# Analyse du trafic cycliste à Paris en fonction de la météo


# Objectifs

Construire un mini-projet d’extraction et d’analyse de données autour du trafic cycliste à Paris. L'objectif est de :   
    ● Comprendre et appliquer le concept d’ETL      
    ● Explorer des données ouvertes (open data)     
    ● Croiser deux sources de données (API + scraping)      
    ● Stocker les données dans une base SQL (SQLite3)       
    ● Réaliser une visualisation (streamlit) et/ou un modèle simple     


# Partie 1 – Introduction à l’ETL

## Qu'est-ce qu'un ETL ?    

● Extract : récupérer des données depuis une source (API, fichier, web, BDD, etc.). Les données sont hétérogènes et non nettoyées ! 

● Transform : nettoyer, reformater, enrichir les données. C'est l'étape clé pour garantir la qualité des données.
        - suppression des doublons      
        - gestion des valeurs manquantes             
        - changement de formats (dates, devises…)       
        - jointures entre tables            
        - agrégations (somme, moyenne…)
        - règles métier (ex : statut client)        
        
● Load : stocker les données dans un format structuré (CSV, DB, etc.)           


# Partie 2 – Le projet

Comment la météo influence-t-elle l'utilisation des pistes cyclables à Paris ?

Livrables attendus à la fin de la journée :     
● Une base SQLite contenant les données croisées vélo + météo       
● Un notebook ou script de traitement (ETL)             
● Facultatif : Une visualisation avec Streamlit (température vs fréquentation)          
● Facultatif : un mini modèle prédictif (trafic vélo en fonction de la météo ?)         

## Etape 1 : Extraction des Données

### 1. Les données météorologiques de la ville de Paris

Dans un premier temps, nous allons avec la méthode du scraping, rechercher les données météo de la Ville de Paris via le site 
https://www.meteociel.fr/climatologie/obs_villes.php



In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.meteociel.fr/climatologie/obs_villes.php?code=7156"  # Paris - Montsouris

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
}

r = requests.get(url, headers=headers, timeout=30)
r.raise_for_status()

# Meteociel peut être en ISO-8859-1 / Windows-1252 selon les pages
# -> on force une valeur courante si besoin
r.encoding = r.apparent_encoding or "ISO-8859-1"

soup = BeautifulSoup(r.text, "html.parser")


In [ ]:
# Récupération de toutes les tables
tables = soup.find_all("table")
print("Nombre de tables trouvées :", len(tables))
print(tables)

Nombre de tables trouvées : 8
[<table align="center" border="0" width="1050">
<tbody>
<tr>
<td class="texte" colspan="2" height="81"><div align="center"><img height="81" src="//static.meteociel.fr/obs3.png" width="1078"/></div></td>
</tr>
<tr class="texte">
<td class="texte" rowspan="2" valign="top" width="160"><table cellpadding="0" cellspacing="0">
<tbody>
<tr valign="top">
<td class="texte" height="1073" width="161">
<div class="nav">
<p><a href="/">Accueil</a><span> - </span><span id="menu_nb_visiteurs">
                        12515 
                        visiteurs </span></p>
<ul id="menu">
<li id="menu_utilisateurs">
<h3 id="menutitre1">Menu Utilisateur</h3>
<div id="menu1" style="display: block">
<ul>
<li class="menu_notice"><a href="/temps-reel/inscription.php">Inscription</a></li>
<li class="menu_notice"><a href="/connexion.php">Se connecter</a></li>
</ul>
<ul>
<li><a href="https://play.google.com/store/apps/details?id=com.meteociel.fr&amp;hl=fr">Meteociel Android</a></li>


In [56]:
# on regarde les lignes des tables pour voir leur contenu
for i, t in enumerate(tables):  
    preview = t.get_text(" ", strip=True)[:360] ## on limite l'affichage à 360 caractères jusqu'à l'espace le plus proche
    print(i, preview)

0 Accueil - 13136 
                        visiteurs Menu Utilisateur Inscription Se connecter Meteociel Android Meteociel iOS Héberger image Forums Meteociel Tchat météo (6) Temps réel Observations Poster vos obs. Détails des obs. Galerie des obs. Carte des photos Partager une photo Température Température min Température max Température mer Point de rosée Wi
1 Accueil - 13136 
                        visiteurs Menu Utilisateur Inscription Se connecter Meteociel Android Meteociel iOS Héberger image Forums Meteociel Tchat météo (6) Temps réel Observations Poster vos obs. Détails des obs. Galerie des obs. Carte des photos Partager une photo Température Température min Température max Température mer Point de rosée Wi
2 • Climatologie mensuelle pour une ville sur tableau et graphiques. Données mensuelles de décembre 2025 pour Paris - Montsouris (75) ( 75 m ) [ Version imprimable ] - [ Fiche station ] - [ Normales 1991-2020 et records ] - [ Comparer avec normales ] N! Station : [ France ]

In [ ]:
# La table météo journalière est l'index 5 car on voit clairement l'entête
target = tables[5]  # <-- la table des données météo journalières

matrix = []
for tr in target.find_all("tr"): ## parcourir les lignes et trouver les cellules avec th ou td qui contiennent les données 
    row = [cell.get_text(" ", strip=True) for cell in tr.find_all(["th", "td"])] ## extraire le texte de chaque cellule avec séparation par un espace
    if row:
        matrix.append(row)

matrix[:5]  # aperçu des 5 premières lignes        

[['Jour',
  'Température max.',
  'Température min.',
  'Précipitations 24h',
  'Ensoleillement',
  ''],
 ['Lun. 1', '8 °C', '1.5 °C', '0.2 mm', '0.7 h', ''],
 ['Mar. 2', '10.1 °C', '7 °C', '0 mm (Tr)', '0.3 h', ''],
 ['Mer. 3', '10.4 °C', '6.4 °C', '0 mm', '2 h', ''],
 ['Jeu. 4', '9.6 °C', '5.5 °C', '1 mm', '1.5 h', '']]

In [ ]:
# pour éviter les soucis si certaines lignes ont moins de colonnes
max_len = max(len(r) for r in matrix) # trouver la longueur maximale des lignes
matrix = [r + [""] * (max_len - len(r)) for r in matrix] # compléter les lignes plus courtes avec des chaînes vides

df = pd.DataFrame(matrix[1:], columns=matrix[0])

df.head()

,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,
0,Lun. 1,8 °C,1.5 °C,0.2 mm,0.7 h,
1,Mar. 2,10.1 °C,7 °C,0 mm (Tr),0.3 h,
2,Mer. 3,10.4 °C,6.4 °C,0 mm,2 h,
3,Jeu. 4,9.6 °C,5.5 °C,1 mm,1.5 h,
4,Ven. 5,9.5 °C,1.6 °C,2.6 mm,2.8 h,


In [18]:
print(df.columns)
print(df.tail(3))

df


Index(['Jour', 'Température max.', 'Température min.', 'Précipitations 24h',
       'Ensoleillement', ''],
      dtype='object')
       Jour Température max. Température min. Précipitations 24h  \
21  Lun. 22           9.9 °C           7.9 °C          0 mm (Tr)   
22  Mar. 23              ---           5.8 °C                ---   
23                   11.7 °C           7.6 °C              39 mm   

   Ensoleillement    
21            0 h    
22                   
23         31.3 h    


,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,
0,Lun. 1,8 °C,1.5 °C,0.2 mm,0.7 h,
1,Mar. 2,10.1 °C,7 °C,0 mm (Tr),0.3 h,
2,Mer. 3,10.4 °C,6.4 °C,0 mm,2 h,
3,Jeu. 4,9.6 °C,5.5 °C,1 mm,1.5 h,
4,Ven. 5,9.5 °C,1.6 °C,2.6 mm,2.8 h,
5,Sam. 6,13.6 °C,5.8 °C,2 mm,3.2 h,
6,Dim. 7,14.6 °C,10.5 °C,9.3 mm,0 h,
7,Lun. 8,15.9 °C,13.3 °C,0 mm,2.5 h,
8,Mar. 9,15.5 °C,9.2 °C,1.2 mm,6.9 h,
9,Mer. 10,14.4 °C,11.7 °C,0.2 mm,0.5 h,


Les données qui ont été scrapés précédemment ne correspondent qu'au mois de décembre 2025. Pour avoir toute l'année 2025 par exemple, on va boucler sur les 12 derniers mois et appelé l'url avec les apramètres annne=2025 et mois = 1...12 (Meteociel les utilise bien sur obs_villes.php)

In [16]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.meteociel.fr/climatologie/obs_villes.php"
headers = {"User-Agent": "Mozilla/5.0"}

params = {"code": "7156", "annee": "2025", "mois": "1"}
r = requests.get(base_url, params=params, headers=headers, timeout=30)
r.raise_for_status()
r.encoding = r.apparent_encoding or "ISO-8859-1"

soup = BeautifulSoup(r.text, "html.parser")
tables = soup.find_all("table")
print("Tables:", len(tables))

for i, t in enumerate(tables[:12]):
    print(i, t.get_text(" ", strip=True)[:160])


Tables: 8
0 Accueil - 13822 
                        visiteurs Menu Utilisateur Inscription Se connecter Meteociel Android Meteociel iOS Héberger image Forums Meteociel Tch
1 Accueil - 13822 
                        visiteurs Menu Utilisateur Inscription Se connecter Meteociel Android Meteociel iOS Héberger image Forums Meteociel Tch
2 • Climatologie mensuelle pour une ville sur tableau et graphiques. Données mensuelles de janvier 2025 pour Paris - Montsouris (75) ( 75 m ) [ Version imprimable
3 Données mensuelles de janvier 2025 pour Paris - Montsouris (75) ( 75 m ) [ Version imprimable ] - [ Fiche station ] - [ Normales 1991-2020 et records ] - [ Comp
4 
5 Jour Température max. Température min. Précipitations 24h Ensoleillement Mer. 1 8.8 °C 4 °C 12.9 mm 0 h Jeu. 2 5 °C 3 °C 7.3 mm 0 h Ven. 3 4.7 °C -0.3 °C 0 mm 7
6 Statistiques du mois Jours de chaleur (Tmax >= 25°C) 0 Jours de forte chaleur (Tmax >= 30°C) 0 Jours de très forte chaleur (Tmax >= 35°C) 0 Jours avec nuit trop
7 Meteoci

In [88]:
### on réécrit tout le code pour pouvoir récupérer sur l'ensemble des mois de l'année

import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.meteociel.fr/climatologie/obs_villes.php"
headers = {"User-Agent": "Mozilla/5.0"}

code_paris = 7156
annee = 2025

dfs = []

for mois in range(1, 13):
    params = {"code": str(code_paris), "annee": str(annee), "mois": str(mois)}
    r = requests.get(base_url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    r.encoding = r.apparent_encoding or "ISO-8859-1"

    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")

    if len(tables) <= 5:
        print(f"[WARN] Pas assez de tables pour {annee}-{mois:02d} (len={len(tables)})")
        continue

    # table météo journalière (validée à la main)
    target = tables[5]

    matrix = []
    for tr in target.find_all("tr"):
        row = [cell.get_text(' ', strip=True) for cell in tr.find_all(['th', 'td'])]
        if row:
            matrix.append(row)

    if len(matrix) < 2:
        print(f"[WARN] Table vide pour {annee}-{mois:02d}")
        continue

    # aligner les lignes pour éviter les colonnes décalées
    max_len = max(len(r) for r in matrix)
    matrix = [r + [""] * (max_len - len(r)) for r in matrix]

    df = pd.DataFrame(matrix[1:], columns=matrix[0])

    # garder seulement les lignes "jours" (évite Statistiques du mois, etc.)
    jours = ("Lun.", "Mar.", "Mer.", "Jeu.", "Ven.", "Sam.", "Dim.")
    if "Jour" in df.columns:
        df = df[df["Jour"].astype(str).str.startswith(jours, na=False)].copy()

    df["annee"] = annee
    df["mois"] = mois

    # enlever colonnes dupliquées si jamais ça arrive
    df.columns = df.columns.astype(str)
    df = df.loc[:, ~df.columns.duplicated()]

    # stocker en mémoire
    dfs.append(df)

    # sauvegarde CSV brut du mois
    out = f"meteo_paris_{annee}_{mois:02d}_brut.csv"
    df.to_csv(out, index=False, encoding="utf-8")
    print(f"[INFO] {out} -> {len(df)} lignes")

    time.sleep(1)

# concat année complète + sauvegarde
if dfs:
    df_annee = pd.concat(dfs, ignore_index=True)
    df_annee.to_csv(f"meteo_paris_{annee}_brut.csv", index=False, encoding="utf-8")
    print(f"[INFO] meteo_paris_{annee}_brut.csv -> {len(df_annee)} lignes")
else:
    print("[WARN] dfs est vide, aucun fichier annuel créé")


[INFO] meteo_paris_2025_01_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_02_brut.csv -> 28 lignes
[INFO] meteo_paris_2025_03_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_04_brut.csv -> 30 lignes
[INFO] meteo_paris_2025_05_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_06_brut.csv -> 30 lignes
[INFO] meteo_paris_2025_07_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_08_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_09_brut.csv -> 30 lignes
[INFO] meteo_paris_2025_10_brut.csv -> 31 lignes
[INFO] meteo_paris_2025_11_brut.csv -> 30 lignes
[INFO] meteo_paris_2025_12_brut.csv -> 23 lignes
[INFO] meteo_paris_2025_brut.csv -> 357 lignes


In [89]:
print("Nb mois récupérés:", len(dfs))

Nb mois récupérés: 12


### 2. Les données comptage vélo de la ville de Paris

Les données compteurs de vélos pour le site de comptage nommé 36 quai de Grenelle: https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/information/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name


In [44]:
import requests
import pandas as pd

base = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptage-velo-donnees-compteurs/records"

## on va sélectionner uniquement les compteurs contenant "grenelle" dans leur nom pour avoir celui du 36 quai de grenelle
params = {
    "select": "name",
    "where": "name like '%grenelle%'", 
    "group_by": "name",
    "limit": 100
}

r = requests.get(base, params=params)
r.raise_for_status()
data = r.json()

compteur=pd.DataFrame(data.get("results", [])).name
compteur

0    36 quai de Grenelle
Name: name, dtype: object

In [48]:
site_name=compteur.iloc[0]
site_name

'36 quai de Grenelle'

In [49]:
import requests
import pandas as pd
from io import StringIO

date_min = "2025-01-01T00:00:00+00:00"
date_max = "2025-12-23T23:59:59+00:00"

dataset = "comptage-velo-donnees-compteurs"
base_export = f"https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/{dataset}/exports/csv"



where = (
    f'name="{site_name}" '
    f'AND date >= "{date_min}" '
    f'AND date <= "{date_max}"'
)
params = {
    "select": "id, name, id_compteur, nom_compteur, sum_counts, date",
    "where": where,
    "order_by": "date",
    }

r = requests.get(base_export, params=params, timeout=120)
r.raise_for_status()


# lire le CSV exporté (ODS peut être en ; ou , selon config -> sep=None auto)
df_velo_2025 = pd.read_csv(StringIO(r.text), sep=None, engine="python")
df_velo_2025["date"] = pd.to_datetime(df_velo_2025["date"], utc=True)

print(df_velo_2025.shape)
df_velo_2025.to_csv("velo_36_quai_de_grenelle_2025_brut.csv", index=False, encoding="utf-8")

(16916, 6)


In [50]:
df_velo_2025.head()

,﻿id,name,id_compteur,nom_compteur,sum_counts,date
0,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,111,2025-01-01 00:00:00+00:00
1,100056330,36 quai de Grenelle,100056330-103056330,36 quai de Grenelle SO-NE,12,2025-01-01 00:00:00+00:00
2,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,57,2025-01-01 01:00:00+00:00
3,100056330,36 quai de Grenelle,100056330-103056330,36 quai de Grenelle SO-NE,34,2025-01-01 01:00:00+00:00
4,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,65,2025-01-01 02:00:00+00:00


## Etape 2 :Transformation et nettoyage des données

### Les données brutes météo de Paris

In [165]:
import pandas as pd
df = pd.read_csv('meteo_paris_2025_brut.csv', sep=',', engine='python')
df.head()

,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,Unnamed: 5,annee,mois,Enneigement
0,Mer. 1,8.8 °C,4 °C,12.9 mm,0 h,NaN,2025,1,NaN
1,Jeu. 2,5 °C,3 °C,7.3 mm,0 h,NaN,2025,1,NaN
2,Ven. 3,4.7 °C,-0.3 °C,0 mm,7.1 h,NaN,2025,1,NaN
3,Sam. 4,11 °C,-3.6 °C,9.7 mm,4.1 h,NaN,2025,1,NaN
4,Dim. 5,12.8 °C,0.9 °C,2 mm,0.1 h,NaN,2025,1,NaN


In [166]:
### on modifie la colonne date pour être en datetime et extraire jour, mois, année
# garder seulement le numéro du jour (Mer. 1 -> 1)
df["Jour"] = df["Jour"].astype(str).str.split().str[-1]
# convertir en int (nullable si jamais il y a un souci)
df["Jour"] = pd.to_numeric(df["Jour"], errors="coerce").astype("Int64")
df.head()


,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,Unnamed: 5,annee,mois,Enneigement
0,1,8.8 °C,4 °C,12.9 mm,0 h,NaN,2025,1,NaN
1,2,5 °C,3 °C,7.3 mm,0 h,NaN,2025,1,NaN
2,3,4.7 °C,-0.3 °C,0 mm,7.1 h,NaN,2025,1,NaN
3,4,11 °C,-3.6 °C,9.7 mm,4.1 h,NaN,2025,1,NaN
4,5,12.8 °C,0.9 °C,2 mm,0.1 h,NaN,2025,1,NaN


In [167]:
# construire une vraie date
df["date"] = pd.to_datetime(
    dict(year=df["annee"], month=df["mois"], day=df["Jour"]),
    errors="coerce"
)

# mettre la date en première colonne
cols = ["date"] + [c for c in df.columns if c != "date"]
df = df[cols]

df.head()

,date,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,Unnamed: 5,annee,mois,Enneigement
0,2025-01-01,1,8.8 °C,4 °C,12.9 mm,0 h,NaN,2025,1,NaN
1,2025-01-02,2,5 °C,3 °C,7.3 mm,0 h,NaN,2025,1,NaN
2,2025-01-03,3,4.7 °C,-0.3 °C,0 mm,7.1 h,NaN,2025,1,NaN
3,2025-01-04,4,11 °C,-3.6 °C,9.7 mm,4.1 h,NaN,2025,1,NaN
4,2025-01-05,5,12.8 °C,0.9 °C,2 mm,0.1 h,NaN,2025,1,NaN


In [168]:
# s'assurer qu'on a bien une date valide
df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.normalize()

In [169]:
# Température max -> float
df["Température max."] = (
    df["Température max."].astype(str)
      .str.replace("°C", "", regex=False)
      .str.replace(",", ".", regex=False)
      .str.strip()
)
df["Température max."] = pd.to_numeric(df["Température max."], errors="coerce")

# Température min -> float
df["Température min."] = (
    df["Température min."].astype(str)
      .str.replace("°C", "", regex=False)
      .str.replace(",", ".", regex=False)
      .str.strip()
)
df["Température min."] = pd.to_numeric(df["Température min."], errors="coerce")

# Précipitations 24h -> float
df["Précipitations 24h"]= (
    df["Précipitations 24h"].astype(str)
      .str.replace("mm", "", regex=False)
      .str.replace(",", ".", regex=False)
      .str.strip()
)
df["Précipitations 24h"] = pd.to_numeric(df["Précipitations 24h"], errors="coerce")

# Ensoleillement -> float
df["Ensoleillement"]= (
    df["Ensoleillement"].astype(str)
      .str.replace("h", "", regex=False)
      .str.replace(",", ".", regex=False)
      .str.strip()
)
df["Ensoleillement"] = pd.to_numeric(df["Ensoleillement"], errors="coerce")

In [170]:
df.head()

,date,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,Unnamed: 5,annee,mois,Enneigement
0,2025-01-01,1,8.8,4.0,12.9,0.0,NaN,2025,1,NaN
1,2025-01-02,2,5.0,3.0,7.3,0.0,NaN,2025,1,NaN
2,2025-01-03,3,4.7,-0.3,0.0,7.1,NaN,2025,1,NaN
3,2025-01-04,4,11.0,-3.6,9.7,4.1,NaN,2025,1,NaN
4,2025-01-05,5,12.8,0.9,2.0,0.1,NaN,2025,1,NaN


In [171]:
df["Unnamed: 5"].unique() ## que des valeurs vides => supprision de la colonne

array([nan])

In [172]:
df = df.drop(columns=["Unnamed: 5"])

In [173]:
## colonne enneigement 
df["Enneigement"].unique()

array([nan, '0 cm', '1 cm', '3 cm'], dtype=object)

In [174]:
# Enneigement -> float
df["Enneigement"]= (
    df["Enneigement"].astype(str)
      .str.replace("cm", "", regex=False)
      .str.replace(",", ".", regex=False)
      .str.replace("nan", "0", regex=False)
      .str.strip()
)
df["Enneigement"] = pd.to_numeric(df["Enneigement"], errors="coerce")

In [175]:
print (df.dtypes)

date                  datetime64[ns]
Jour                           Int64
Température max.             float64
Température min.             float64
Précipitations 24h           float64
Ensoleillement               float64
annee                          int64
mois                           int64
Enneigement                    int64
dtype: object


In [176]:
df.head()


,date,Jour,Température max.,Température min.,Précipitations 24h,Ensoleillement,annee,mois,Enneigement
0,2025-01-01,1,8.8,4.0,12.9,0.0,2025,1,0
1,2025-01-02,2,5.0,3.0,7.3,0.0,2025,1,0
2,2025-01-03,3,4.7,-0.3,0.0,7.1,2025,1,0
3,2025-01-04,4,11.0,-3.6,9.7,4.1,2025,1,0
4,2025-01-05,5,12.8,0.9,2.0,0.1,2025,1,0


In [177]:
# Dernière vérification le nombre de jour par mois
df.groupby("mois")["Jour"].count()

mois
1     31
2     28
3     31
4     30
5     31
6     30
7     31
8     31
9     30
10    31
11    30
12    23
Name: Jour, dtype: Int64

In [178]:
# regarder s'il y a des dates en double
df.duplicated(subset=["date"]).sum()


np.int64(0)

In [179]:
df_meteo = pd.DataFrame(df).copy()

# optionnel : renommer des colonnes météo pour éviter les espaces / accents en SQL
df_meteo = df_meteo.rename(columns={
    "Température max.": "tmax_c",
    "Température min.": "tmin_c",
    "Précipitations 24h": "precip_24h",
    "Ensoleillement": "sun_h",
    "Enneigement":"snow_cm"
})

In [180]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        357 non-null    datetime64[ns]
 1   Jour        357 non-null    Int64         
 2   tmax_c      356 non-null    float64       
 3   tmin_c      357 non-null    float64       
 4   precip_24h  327 non-null    float64       
 5   sun_h       356 non-null    float64       
 6   annee       357 non-null    int64         
 7   mois        357 non-null    int64         
 8   snow_cm     357 non-null    int64         
dtypes: Int64(1), datetime64[ns](1), float64(4), int64(3)
memory usage: 25.6 KB


Les données météo sont prètes à être utilisées pour l'année 2025 !

### Les données Vélo


In [181]:
dfVelo = pd.read_csv('velo_36_quai_de_grenelle_2025_brut.csv', sep=',', engine='python')
dfVelo.head()

,id,name,id_compteur,nom_compteur,sum_counts,date
0,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,111,2025-01-01 00:00:00+00:00
1,100056330,36 quai de Grenelle,100056330-103056330,36 quai de Grenelle SO-NE,12,2025-01-01 00:00:00+00:00
2,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,57,2025-01-01 01:00:00+00:00
3,100056330,36 quai de Grenelle,100056330-103056330,36 quai de Grenelle SO-NE,34,2025-01-01 01:00:00+00:00
4,100056330,36 quai de Grenelle,100056330-104056330,36 quai de Grenelle NE-SO,65,2025-01-01 02:00:00+00:00


In [182]:
dfVelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16916 entries, 0 to 16915
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            16916 non-null  int64 
 1   name          16916 non-null  object
 2   id_compteur   16916 non-null  object
 3   nom_compteur  16916 non-null  object
 4   sum_counts    16916 non-null  int64 
 5   date          16916 non-null  object
dtypes: int64(2), object(4)
memory usage: 793.1+ KB


In [183]:
## on va regrouper par date pour avoir le total journalier
dfVelo["date"] = pd.to_datetime(dfVelo["date"])
# créer une colonne jour (sans l'heure)
dfVelo["jour"] = dfVelo["date"].dt.date

# groupby par jour → somme des comptages
dfVelo_journalier = (
    dfVelo
    .groupby("jour", as_index=False)["sum_counts"]
    .sum()
)

In [184]:
dfVelo_journalier.head()

,jour,sum_counts
0,2025-01-01,1329
1,2025-01-02,1335
2,2025-01-03,2320
3,2025-01-04,1448
4,2025-01-05,1502


In [185]:
dfVelo_journalier = dfVelo_journalier.rename(columns={
    "jour": "date",
    "sum_counts": "nb_velos"
})

In [186]:
# --- côté vélo : enlever timezone + garder la date (jour) ---
dfVelo_journalier["date"] = pd.to_datetime(dfVelo_journalier["date"], utc=True, errors="coerce")
dfVelo_journalier["date"] = dfVelo_journalier["date"].dt.tz_convert(None).dt.normalize()  # -> datetime64[ns] à 00:00


In [187]:
dfVelo_journalier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      356 non-null    datetime64[ns]
 1   nb_velos  356 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.7 KB


In [188]:
dfVelo_journalier.head()

,date,nb_velos
0,2025-01-01,1329
1,2025-01-02,1335
2,2025-01-03,2320
3,2025-01-04,1448
4,2025-01-05,1502


## Etape 3 : Création de la base SQLite contenant les données croisées vélo + météo 

In [ ]:
import pandas as pd

df_merge = pd.merge(dfVelo_journalier, df_meteo, on="date", how="inner")

In [190]:
df_merge.head()

,date,nb_velos,Jour,tmax_c,tmin_c,precip_24h,sun_h,annee,mois,snow_cm
0,2025-01-01,1329,1,8.8,4.0,12.9,0.0,2025,1,0
1,2025-01-02,1335,2,5.0,3.0,7.3,0.0,2025,1,0
2,2025-01-03,2320,3,4.7,-0.3,0.0,7.1,2025,1,0
3,2025-01-04,1448,4,11.0,-3.6,9.7,4.1,2025,1,0
4,2025-01-05,1502,5,12.8,0.9,2.0,0.1,2025,1,0


In [191]:
import sqlite3
# Créer une Connexion SQLite

db_path = "paris_velo_meteo_2025.sqlite"
con = sqlite3.connect(db_path)

In [192]:
# Ecriture des tables dans la base SQLite

df_meteo.to_sql("meteo_paris_2025", con, if_exists="replace", index=False)
dfVelo_journalier.to_sql("velo_36_quai_grenelle_2025", con, if_exists="replace", index=False)
df_merge.to_sql("velo_meteo_2025", con, if_exists="replace", index=False)

356

In [193]:
# On peut Indexer directement sur les dates (pour aller plus vite)

con.execute("CREATE INDEX IF NOT EXISTS idx_meteo_date ON meteo_paris_2025(date);")
con.execute("CREATE INDEX IF NOT EXISTS idx_velo_date ON velo_36_quai_grenelle_2025(date);")
con.execute("CREATE INDEX IF NOT EXISTS idx_vm_date ON velo_meteo_2025(date);")
con.commit()

In [194]:
# Vérficication

print(pd.read_sql_query("SELECT COUNT(*) AS nb_lignes FROM meteo_paris_2025;", con))
print(pd.read_sql_query("SELECT COUNT(*) AS nb_lignes FROM velo_36_quai_grenelle_2025;", con))
print(pd.read_sql_query("SELECT COUNT(*) AS nb_lignes FROM velo_meteo_2025;", con))
print(pd.read_sql_query("SELECT MIN(date) AS debut, MAX(date) AS fin FROM velo_meteo_2025;", con))


   nb_lignes
0        357
   nb_lignes
0        356
   nb_lignes
0        356
                 debut                  fin
0  2025-01-01 00:00:00  2025-12-22 00:00:00


In [197]:
# aperçu
print(pd.read_sql_query("SELECT * FROM velo_meteo_2025 ORDER BY date LIMIT 5;", con))

con.close()

                  date  nb_velos  Jour  tmax_c  tmin_c  precip_24h  sun_h  \
0  2025-01-01 00:00:00      1329     1     8.8     4.0        12.9    0.0   
1  2025-01-02 00:00:00      1335     2     5.0     3.0         7.3    0.0   
2  2025-01-03 00:00:00      2320     3     4.7    -0.3         0.0    7.1   
3  2025-01-04 00:00:00      1448     4    11.0    -3.6         9.7    4.1   
4  2025-01-05 00:00:00      1502     5    12.8     0.9         2.0    0.1   

   annee  mois  snow_cm  
0   2025     1        0  
1   2025     1        0  
2   2025     1        0  
3   2025     1        0  
4   2025     1        0  


In [198]:
print(f"La Base SQLite créée : {db_path}")

La Base SQLite créée : paris_velo_meteo_2025.sqlite


## Etape 4: Visualisation des données 

### Nombre de vélos par rapport aux précipitations


In [214]:
# on va enregistrer les figures pour créer un rapport en streamlit
from pathlib import Path

out_dir = Path("figures")
out_dir.mkdir(exist_ok=True)

In [215]:
df = df_merge.copy()
df = df.dropna(subset=["date", "nb_velos", "precip_24h"]).copy()

In [217]:
import plotly.graph_objects as go
import plotly.io as pio
fig1 = go.Figure()

fig1.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["nb_velos"],
        name="Vélos / jour",
        mode="lines"
    )
)

# pluie (axe Y droit) en barres
fig1.add_trace(
    go.Bar(
        x=df["date"],
        y=df["precip_24h"],
        name="Pluie (mm)",
        yaxis="y2",
        opacity=0.5
    )
)


fig1.update_layout(
    title="Vélos vs précipitations – 36 quai de Grenelle (Paris, 2025)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Nombre de vélos / jour"),
    yaxis2=dict(
        title="Précipitations (mm / 24h)",
        overlaying="y",
        side="right"
    ),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    bargap=0,
)

fig1.show()
pio.write_json(fig1, out_dir / "fig_velo_precipitation_time.json")

In [225]:
import plotly.express as px
fig2 = px.scatter(
    df,
    x="precip_24h",
    y="nb_velos",
    trendline="ols",
    hover_data=["date"],
    labels={"precip_24h": "Précipitations (mm / 24h)", "nb_velos": "Nombre de vélos / jour"},
    title="Impact des précipitations sur le nombre de vélos (Paris, 2025)"
)
fig2.show()
pio.write_json(fig2, out_dir / "fig_velo_pluie.json")


### Nombre de vélos par rapport à la température

In [219]:
import plotly.graph_objects as go


# température moyenne du jour
df["temp_moy"] = (df["tmax_c"] + df["tmin_c"]) / 2

df = df.dropna(subset=["nb_velos", "temp_moy"])


fig3 = go.Figure()

# vélos
fig3.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["nb_velos"],
        name="Vélos / jour",
        mode="lines"
    )
)

# température
fig3.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["temp_moy"],
        name="Température moyenne (°C)",
        yaxis="y2",
        mode="lines"
    )
)

fig3.update_layout(
    title="Évolution du trafic vélo et de la température (Paris, 2025)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Nombre de vélos / jour"),
    yaxis2=dict(
        title="Température moyenne (°C)",
        overlaying="y",
        side="right"
    ),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
)

fig3.show()
pio.write_json(fig3, out_dir / "fig_velo_temperature_time.json")

In [224]:
import plotly.express as px

fig4 = px.scatter(
    df,
    x="temp_moy",
    y="nb_velos",
    trendline="ols",
    labels={
        "temp_moy": "Température moyenne (°C)",
        "nb_velos": "Nombre de vélos / jour",
    },
    title="Relation entre température et usage du vélo (Paris, 2025)"
)

fig4.show()
pio.write_json(fig4, out_dir / "fig_velo_temperature.json")
